<a href="https://colab.research.google.com/github/ManavMoitra/tensorflow-2.3.0/blob/master/Transferlearning_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import GlobalAveragePooling2D,Dense
from tensorflow.keras.models import Model
import os
from tensorflow.keras import layers,models
import PIL
from PIL import Image

In [85]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [86]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions

In [87]:
from tensorflow.keras.applications import VGG16

In [88]:
img_rows,img_cols=224,224

In [89]:
num_classes=10

In [90]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_data_path = '/content/drive/MyDrive/monkey_species/training/training'
validation_data_path = '/content/drive/MyDrive/monkey_species/validation/validation'
monkey_species = os.listdir(train_data_path)
img_width, img_height = 224, 224
batch_size = 4

print("Number of Categories:", len(monkey_species))
print("Categories: ", monkey_species)

Number of Categories: 10
Categories:  ['n6', 'n9', 'n0', 'n4', 'n7', 'n3', 'n8', 'n5', 'n2', 'n1']


In [91]:
label = ["Mantled Howler","Patas Monkey","Bald Uakari",
         "Japanese Macaque","Pygmy Marmoset","White Headed Capuchin",
         "Silvery Marmoset","Ommon Squirrel Monkey",
         "Black Headed Night Monkey","Nilgiri Langur"]

In [92]:
train_datagen = ImageDataGenerator(
    rotation_range = 30,
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 1110 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [93]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

In [94]:
model_base = VGG16(weights='imagenet',include_top=False)

x = model_base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512,activation='relu')(x)
predictions = layers.Dense(int(len(train_generator.class_indices.keys())) ,activation='softmax')(x)
Vgg16_model = models.Model(inputs= model_base.input, outputs=predictions)

58892288/58889256 [==============================] - 0s 0us/step


In [95]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras import optimizers

In [96]:
checkpoint=ModelCheckpoint(r"/content/drive/My Drive/monkey_species/modelcheckpoint",
                           monitor='val_loss',
                           mode='min',
                           save_best_only=True,
                           verbose=1)

In [97]:
Vgg16_model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9) ,metrics=['accuracy'])
history = Vgg16_model.fit(train_generator, epochs=30, shuffle=True, verbose=1, validation_data=validation_generator)

Epoch 1/30
278/278 [==============================] - 390s 1s/step - loss: 2.1933 - accuracy: 0.1919 - val_loss: 1.9791 - val_accuracy: 0.3235
Epoch 2/30
278/278 [==============================] - 55s 199ms/step - loss: 1.8737 - accuracy: 0.3423 - val_loss: 1.8507 - val_accuracy: 0.3125
Epoch 3/30
278/278 [==============================] - 55s 199ms/step - loss: 1.5217 - accuracy: 0.4739 - val_loss: 1.1176 - val_accuracy: 0.6250
Epoch 4/30
278/278 [==============================] - 55s 199ms/step - loss: 1.1464 - accuracy: 0.6468 - val_loss: 1.0506 - val_accuracy: 0.6434
Epoch 5/30
278/278 [==============================] - 56s 201ms/step - loss: 0.7832 - accuracy: 0.7459 - val_loss: 0.7650 - val_accuracy: 0.7574
Epoch 6/30
278/278 [==============================] - 56s 201ms/step - loss: 0.5546 - accuracy: 0.8198 - val_loss: 0.6169 - val_accuracy: 0.8125
Epoch 7/30
278/278 [==============================] - 56s 201ms/step - loss: 0.4057 - accuracy: 0.8712 - val_loss: 0.4940 - val_accu

In [98]:
tf.keras.models.save_model(Vgg16_model,'vgg16_model.hdf5')

In [99]:
from google.colab import drive
drive.mount('/content/drive')                          

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [100]:
!pip install streamlit
!pip install pyngrok==4.1.1

In [106]:
%%writefile transfer.py
import PIL
import streamlit as st 
from PIL import Image
import tensorflow as tf
import numpy as np

st.set_option("deprecation.showfileUploaderEncoding",False)
@st.cache(allow_output_mutation=True)
def load_model():
  model=tf.keras.models.load_model('/content/vgg16_model.hdf5')
  return model

model=load_model()
st.write("""
         #MONKEY CLASSIFICATION 
         UPLOAD IMAGES FROM:-
         "Mantled Howler","Patas Monkey","Bald Uakari",
         "Japanese Macaque","Pygmy Marmoset","White Headed Capuchin",
         "Silvery Marmoset","Ommon Squirrel Monkey",
         "Black Headed Night Monkey","Nilgiri Langur"

          """
          )
file=st.file_uploader("please upload a monkey image ",type=['jpg','png'])
import cv2
from PIL import Image,ImageOps
import numpy as np
def import_and_predict(image_data,model):
  size=(224,224)
  image=ImageOps.fit(image_data,size,Image.ANTIALIAS)
  img=np.asarray(image)
  img_reshape=img[np.newaxis,...]
  prediction=model.predict(img_reshape)
  return prediction

if file is None:
  st.text('please upload an image')
else:
  image=Image.open(file)
  st.image(image,use_column_width=True)
  predictions=import_and_predict(image,model)
  class_names=["Mantled Howler","Patas Monkey","Bald Uakari",
               "Japanese Macaque","Pygmy Marmoset","White Headed Capuchin",
               "Silvery Marmoset","Ommon Squirrel Monkey",
               "Black Headed Night Monkey","Nilgiri Langur"]

  string="This image is most likely : "+class_names[np.argmax(predictions)]
  st.success(string) 
 







Overwriting transfer.py


In [107]:
from pyngrok import ngrok

In [108]:
!ngrok authtoken 1ldQo6dNyopiyPolftkveS91I0q_3GW11M62yrJnHNmfZNyfj

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [109]:
!nohup streamlit run transfer.py &

nohup: appending output to 'nohup.out'


In [110]:

url=ngrok.connect(port=8501)
print(url)

t=2020-12-14T06:30:46+0000 lvl=warn msg="failed to start tunnel" pg=/api/tunnels id=43c959ad375a468b err="Your account may not run more than 4 tunnels over a single ngrok client session.\nThe tunnels already running on this session are:\n[http://4499c682970b.ngrok.io https://4499c682970b.ngrok.io http://adc8463cc95c.ngrok.io https://adc8463cc95c.ngrok.io]\n\r\n\r\nERR_NGROK_324\r\n"



PyngrokNgrokHTTPError: ignored